In order to check if EtablissementsScolairesPrives and fr-en-annuaire-education are complete DB 
let's merge them on adress of values as Primary Key.

Once databases will be merged it will be then enriched by further scrapping

TROUVER LES LIENS!

I format and group fr-en-annuaire-education on address

First step is to create Primary Key in fr-en-annuaire-education by concatenation of targeted columns and formating both Primary Key of dabases in order to don't miss a match

In [1]:
import pandas as pd
import re

In [2]:
scrappedDB = pd.read_csv("path2project/Annuaire/EtablissementScolairesPrives/EtablissementsScolairesPrives.csv")

In [3]:
datagouv = pd.read_csv("path2project/Annuaire/EtablissementScolairesPrives/fr-en-annuaire-education.csv")

Check if scrappedDB has values starting by 0

reshape datagouv in order to match scrappeDB by removing special characteres

In [4]:
replacements = {'é': 'e', 'É':'E', 'ê':'e', 'è': 'e', 'œ':'oe', 'â':'a', 'à':'a', 'ô':'o', '…':'', 'ç':'c', 'î':'i', 'Î':'I', 'ï':'i', 'ë':'e', 'û':'u', '\n':' '}
datagouv2 = datagouv.replace(replacements, regex=True)

Select usefull columns from datagouv2 to datagouv3

In [5]:
datagouv3 = datagouv2[['Nom_etablissement', 'Adresse_1', 'Code postal', 'Nom_commune']]
 

Get rid of corrupted rows

In [6]:
datagouv3 = datagouv3.dropna(subset=['Nom_etablissement', 'Adresse_1', 'Code postal', 'Nom_commune'])

Convert Code Postal into an int column

In [7]:
datagouv3['Code postal'] = datagouv3['Code postal'].apply(int)

Format commune "Paris ne Arrondissement" to "Paris n"
(must be runned twice)

In [8]:
datagouv4 = datagouv3
regex = '^(Paris\s+\d+)(e\s+Arrondissement)$'
for i, commune in enumerate(datagouv4['Nom_commune']):
    match = re.search(regex, commune)
    if match:
        datagouv4['Nom_commune'].iloc[i] = datagouv4['Nom_commune'].iloc[i].strip(match.group(2))

C:\Users\morel\AppData\Local\Temp\ipykernel_8312\1826098303.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datagouv4['Nom_commune'].iloc[i] = datagouv4['Nom_commune'].iloc[i].strip(match.group(2))


Convert Code Postal to string

In [9]:
datagouv4['Code postal'] = datagouv4['Code postal'].apply(str)

Create Primary Key 'AdresseEtablissementPrive' by concatenation of "Adresse_1", "Code postal", and "Nom_commune"

In [10]:
datagouv4['AdresseEtablissementPrive'] = datagouv4.apply(lambda row: row['Adresse_1'] + ' ' + row['Code postal'] + ' ' + row['Nom_commune'], axis = 1) 

Slim datagouv4 

In [11]:
datagouv4 = datagouv4.drop(['Adresse_1', 'Code postal', 'Nom_commune'], axis = 1)

datagouv4[]

Raname establishment names as in scrappedDB

In [12]:
datagouv4.rename(columns = {'Nom_etablissement':'NomEtablissementPrive'}, inplace = True)

Search for "AdresseEtablissementPrive" rows starting by 0 

In [13]:
datagouv4[datagouv4['AdresseEtablissementPrive'].apply(lambda x: re.search('^0', x)).notna()]

,NomEtablissementPrive,AdresseEtablissementPrive
52,Ecole elementaire privee La Plume de l'Education,01 rue ERIC TABARLY 93140 Bondy
126,Ecole elementaire privee Jeanne d'Arc Notre-Dame,06 rue du General Colin 78400 Chatou
129,Ecole elementaire privee Sainte Anne,08 rue de la Tour 78362 Montesson
134,Ecole elementaire privee Le Bon Sauveur,06 rue Henri Cloppet 78110 Le Vesinet
137,Ecole elementaire privee Jeanne d'Arc,06 rue Armand Chardron 78110 Le Vesinet
298,Lycee professionnel prive Sports etudes de Par...,0 chemin DES MARMOUSETS 94510 La Queue-en-Brie
330,IME Le Tremplin,09 RUE GEORGES BRUYERE 93000 Bobigny
363,Ecole elementaire privee Saint Symphorien,01 rue du Chanoine Boyer 78000 Versailles
421,Ecole elementaire privee Saint Joseph,02 place de l'Eglise 78700 Conflans-Sainte-Hon...
423,Ecole elementaire privee Blanche de Louvencourt,01 rue de Louvencourt 78160 Marly-le-Roi


Strip first 0 from them

In [14]:
datagouv4['AdresseEtablissementPrive'] = datagouv4['AdresseEtablissementPrive'].apply(lambda x: re.sub('^0', '', x))

Check

In [15]:
datagouv4[datagouv4['AdresseEtablissementPrive'].apply(lambda x: re.search('^0', x)).notna()]

,NomEtablissementPrive,AdresseEtablissementPrive


In [16]:
datagouv4['AdresseEtablissementPrive'].replace('\'', ' ', inplace = True)

group datagouv4 by 'AdresseEtablissementPrive', set 'NomEtablissementPrive' as list

In [17]:
grouped_dg = pd.DataFrame(datagouv4.groupby('AdresseEtablissementPrive')['NomEtablissementPrive'].apply(list))

In [60]:
len(scrappedDB)

972

In [68]:
a  = scrappedDB.set_index(['AdresseEtablissementPrive', 'NomEtablissementPrive'])

In [67]:
scrappedDB['AdresseEtablissementPrive'].duplicated().sum()

203

In [ ]:
datagouv4.set_index(['AdresseEtablissementPrive', 'NomEtablissementPrive'])

In [56]:
datagouv4.set_index(['AdresseEtablissementPrive', 'NomEtablissementPrive'])

,
AdresseEtablissementPrive,NomEtablissementPrive
28 rue Beaurepaire 75010 Paris 10,E.M.PR BEAUREPAIRE 28 rue Beaurepaire
16 rue de Milan 75009 Paris 9,E.P.PR LA TRINITE 16 rue de Milan
80 rue Boileau 75016 Paris 16,E.P.PR LAMAZOU 80 rue Boileau
6 rue Gambetta 78440 Gargenville,Ecole elementaire privee Petite ecole bilingue
33 ter rue Jean Jaures 78540 Vernouillet,Ecole primaire privee Ecole Saint Philippe Neri
...,...
35 rue Saint Blaise 75020 Paris 20,Cours Prive Autograf Formations
104 RUE CASTAGNARY 75015 Paris 15,Institut Nemo
13 rue de la Fontaine au Roi 75011 Paris 11,Ecole Privee de Dietetique et Nutrition Humaine


datagouv4

II group scrappeDB

In [18]:
scrappedDB[scrappedDB['AdresseEtablissementPrive'].apply(lambda row: re.search(r'^0', row)).notna()]

,NomEtablissementPrive,AdresseEtablissementPrive


In [19]:
scrappedDB.dropna(inplace=True)

In [20]:
scrappedDB['AdresseEtablissementPrive'].replace('\'', ' ', inplace = True)

Remove 0 from 'Paris 0n'

In [21]:
scrappedDB['AdresseEtablissementPrive'] = scrappedDB['AdresseEtablissementPrive'].apply(lambda addr: re.sub(r'Paris 0', 'Paris ', addr))

group scrappedDB by 'AdresseEtablissementPrive', set 'NomEtablissementPrive' as list

In [22]:
grouped_sdb = pd.DataFrame(scrappedDB.groupby('AdresseEtablissementPrive')['NomEtablissementPrive'].apply(list))


merge two df on matching AdresseEtablissementPrive values only

In [69]:
df = pd.merge(grouped_dg, grouped_sdb, left_index = True, right_index = True, how = 'inner')

In [71]:
len(df)

253

In [24]:
df['NomEtablissementPrive'] = df.apply(lambda row: row['NomEtablissementPrive_x'] + (row['NomEtablissementPrive_y']), axis = 1)

df.drop(['NomEtablissementPrive_x', 'NomEtablissementPrive_y'], axis = 1, inplace = True)

private grouped_dg from commune values with scrapedDB

In [25]:
dg = grouped_dg[~grouped_dg.index.isin(df.index)]

1393 groups(addresses) were in grouped_dg and not in grouped_sdb

In [26]:
len(grouped_dg)

1393

In [27]:
sdb = grouped_sdb[~grouped_sdb.index.isin(df.index)]

1393 groups(addresses) were in grouped_sdb and not in grouped_dg

In [28]:
len(sdb)

516

In [29]:
adressesEcoles = pd.concat([df, dg, sdb], sort = True)

In [30]:
adressesEcoles.index = adressesEcoles.index.str.upper()

In [52]:
adressesEcoles

,NomEtablissementPrive
AdresseEtablissementPrive,
1 AVENUE RATISBONNE 91000 EVRY-COURCOURONNES,"[Lycee polyvalent Notre-Dame de Sion, College ..."
1 AVENUE DE VERDUN 95300 PONTOISE,"[College Saint-Martin de France, College Prive..."
1 AVENUE DE LA BRETECHE 78150 LE CHESNAY-ROCQUENCOURT,"[College Blanche de Castille, Lycee Blanche de..."
1 BOULEVARD DU NORD 93340 LE RAINCY,"[College Tebrotzassere, College Tebrotzassere]"
1 DOMAINE DU VIGNOIS 77440 ARMENTIERES-EN-BRIE,"[Lycee general prive Beth Myriam, College priv..."
...,...
RUE DU 8 MAI 1945 91120 PALAISEAU,[Lycee Polyvalent Prive Saint Martin]
SITE DU BEL-AIR 51 BOULEVARD DE LA PAIX 78100 SAINT-GERMAIN-EN-LAYE,[Ecole Sup de V - CCI Paris Ile-de-France]
"VOG ACADEMY, HOTEL MARIOTT 17 BOULEVARD SAINT-JACQUES 75014 PARIS 14",[Ecole de Coiffure Formul'Academie]


1909 adresse au total

In [32]:
len(adressesEcoles)

1909

In [33]:
adressesEcoles.to_csv('path2project/Annuaire/adressesEcoles')

In order to find 'denominationUniteLegale' of each establishment I have intented to merge adressesEcoles with a gouvernemental database on adress, however only one match has been found, see below for further information.

In [74]:
sirene = pd.read_csv("path2project/Annuaire/EtablissementScolairesPrives/etablissementsSirene.csv", low_memory = False)

In [75]:
sirene.columns

Index(['siren', 'nic', 'siret', 'statutDiffusionEtablissement',
       'dateCreationEtablissement', 'trancheEffectifsEtablissement',
       'anneeEffectifsEtablissement',
       'activitePrincipaleRegistreMetiersEtablissement',
       'dateDernierTraitementEtablissement', 'etablissementSiege',
       'etatAdministratifUniteLegale', 'statutDiffusionUniteLegale',
       'unitePurgeeUniteLegale', 'dateCreationUniteLegale',
       'categorieJuridiqueUniteLegale', 'denominationUniteLegale',
       'sigleUniteLegale', 'denominationUsuelle1UniteLegale',
       'denominationUsuelle2UniteLegale', 'denominationUsuelle3UniteLegale',
       'sexeUniteLegale', 'nomUniteLegale', 'nomUsageUniteLegale',
       'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale',
       'prenom4UniteLegale', 'prenomUsuelUniteLegale', 'pseudonymeUniteLegale',
       'activitePrincipaleUniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale',
       'identifiantAssociationUniteLegale',
       'econom

In [77]:
len(sirene[(sirene['libelleVoieEtablissement'].isna() == True) & (sirene['complementAdresseEtablissement'].isna()== True)])

26

In [36]:
denominationsSirene = sirene.loc[:, 'denominationUniteLegale':'denominationUsuelle3UniteLegale']

adressesSirene = sirene.loc[:,'numeroVoieEtablissement':'libelleCommuneEtablissement']

siege = sirene['etablissementSiege']

In [37]:
sirene = pd.concat([denominationsSirene, adressesSirene], axis = 1)

In [38]:
sirene['typeVoieEtablissement'].replace(['AV', 'BD', 'CHE', 'ALL', 'PL', 'IMP', 'SQR'], ['AVENUE', 'BOULEVARD', 'CHEMIN', 'ALLEE', 'PLACE', 'IMPASSE', 'SQUARE'], inplace = True)

In [39]:
def sigleMaker(string):
    articles = ['D\'', 'DE', 'DES', 'DU', 'ET','LE','LA']
    words = str(string).split()
    for l in words:
        if l in articles:
                words.remove(l)
    sigle = ''.join(word[0] for word in words)
    print(sigle)
    return(sigle)

In [40]:
nomUniteLegale = []
AdresseEtablissementPrive = []
for i in range(len(sirene)):
     Zip = zip(sirene.columns, sirene.iloc[i])
     sigle = sigleMaker(sirene["denominationUniteLegale"].iloc[i])
     adresse = ''
     for j, k in Zip:
          if sirene.columns.get_loc(j) < len(denominationsSirene.columns) and not pd.isna(k) and k != sigle and k != r'[ND]':
                    uniteLegale = str(k)
          elif sirene.columns.get_loc(j) >= len(denominationsSirene.columns) and sirene.columns.get_loc(j) < len(sirene.columns) and not pd.isna(k) and k!=0 and k != r'[ND]':
               adresse = ' '.join([adresse, str(k)])
     search = ' '.join([uniteLegale, adresse])
     nomUniteLegale.append(uniteLegale)
     AdresseEtablissementPrive.append(adresse)

DAPN
DAPN
DRHMEF
SLADM
INSFDRPLI
ENSIPLILE
INSIVDL
INSIVDL
INSIVDL
INSIVDL
INSIVDL
INSIVDL
EHEESP
ENFP
ENFP
ENFP
ENFP
ENFP
GENDES
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
CCDIRPIF
DZARAFLPNP-I
DZARAFLPNP-I
C
ENSCP
CDEHESAMU
CDUEUPL
CCDRO
CCDRN
CCDRN
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
SU
INLJDLP
AR
IPP
UPC
UPC
UPC
UPC
UPC
UPC
UPC
UPC
UPC
UPC
UPC
UPC
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
CCPU
UP
UP
UP
UP
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UGE
UP
INDSPLLL
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CMLDRI
CDEUP
UP
UP
UP
EPMLD
DTLSP-P
RLP
D

In [41]:
sirene2 = {
    'nomUniteLegale' : nomUniteLegale,
    'AdresseEtablissementPrive' : AdresseEtablissementPrive,
}

In [42]:
sirene2 = pd.DataFrame(sirene2)

In [43]:
#????????????????????????INTERESTING????????????????????????????????

#sirene2['nbrEtablissements'] = sirene2.groupby('nomSirene').transform('count')
    

In [44]:
grouped_sirene = sirene2.groupby('AdresseEtablissementPrive')['nomUniteLegale'].apply(list)

In [45]:
grouped_sirene = pd.DataFrame(grouped_sirene)

In [46]:
AdresseEtablissementPrive = pd.DataFrame({'AdresseEtablissementPrive' : AdresseEtablissementPrive})

In [47]:
siege = pd.concat([AdresseEtablissementPrive, siege], axis = 1)

In [48]:
siege.set_index('AdresseEtablissementPrive', inplace = True)

In [49]:
grouped_sirene = grouped_sirene.merge(siege, left_index = True, right_index = True, how = 'inner')

Merge df and Sirene in order to retrieve 'siege' and 'unite legale' 
Great data volume contrains to merge by chunking 

In [50]:
l = []
for i in adressesEcoles.index:
    for j in grouped_sirene.index:
        if i == j:
            l.append(j) 

print(l)

[' CHEMIN DES MARMOUSETS 94510 LA QUEUE-EN-BRIE']


In [51]:
df2 = pd.DataFrame({})
k = 1001
for i in range(0, 74):
    if i == 0:                              #Unite Legale Must be added
        merge = adressesEcoles.merge(grouped_sirene.iloc[:k], left_index = True, right_index = True, how = 'inner')
        df2 = pd.concat([df2, merge], axis = 0)
        k +=1001
        print(k)
    elif i < 73: 
        j = k-1001
        merge = adressesEcoles.merge(grouped_sirene.iloc[j:k], left_index = True, right_index = True, how = 'inner')
        df2 = pd.concat([df2, merge], axis = 0)
        k +=1000
    else:
        j = k - 1
        k = j + len(grouped_sirene) - 1 
        merge = adressesEcoles.merge(grouped_sirene.iloc[j:k], left_index = True, right_index = True, how = 'inner')
        df2 = pd.concat([df2, merge], axis = 0)

2002


Sirene DB has a great amount of missing values which might be why didn't found only one matching address
looking on Siret might be an option even though scrappedDB doesn't contain this variable